# Using the BackT Strategy Library

This notebook demonstrates how to use pre-built strategies from the `strategies/` module.

## Benefits
- No need to rewrite strategy logic in every notebook
- Consistent implementation across projects
- Easy to compare multiple strategies
- Simple parameter tuning

## Setup

In [ ]:
# Add parent directory to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

# Import BackT
from backt import Backtester, BacktestConfig
from backt.reporting import PerformanceReport

# Import strategies from library
from strategies import (
    ma_crossover_long_only,
    ma_crossover_long_short,
    kalman_ma_crossover_long_only,
    kalman_ma_crossover_long_short
)

import pandas as pd

print("✅ Imports successful!")
print("📚 Available strategies: ma_crossover (long/short), kalman_ma_crossover (long/short)")

## Configuration

In [ ]:
# Define universe
universe = ['SPY', 'QQQ', 'TLT', 'GLD']

# Backtest configuration
config = BacktestConfig(
    start_date='2020-01-01',
    end_date='2023-12-31',
    initial_capital=100000.0,
    allow_short=True,
    max_leverage=2.0,
    verbose=False
)

print(f"Universe: {universe}")
print(f"Period: {config.start_date} to {config.end_date}")
print(f"Initial Capital: ${config.initial_capital:,.0f}")

## Example 1: Single Strategy

In [ ]:
# Define strategy parameters
params = {
    'fast_ma': 20,
    'slow_ma': 50,
    'max_position_size': 0.25
}

# Run backtest
backtester = Backtester(config)
result = backtester.run(
    strategy=ma_crossover_long_only,  # Just pass the function!
    universe=universe,
    strategy_params=params
)

# Generate report
report = PerformanceReport(result, initial_capital=config.initial_capital)
report.print_report()

## Example 2: Compare Multiple Strategies

In [ ]:
# Define strategies to compare
strategies = {
    'MA Long Only': (
        ma_crossover_long_only,
        {'fast_ma': 20, 'slow_ma': 50, 'max_position_size': 0.25}
    ),
    'MA Long-Short': (
        ma_crossover_long_short,
        {'fast_ma': 20, 'slow_ma': 50, 'max_position_size': 0.25}
    ),
    'Kalman Long Only': (
        kalman_ma_crossover_long_only,
        {'Q_fast': 0.01, 'Q_slow': 0.001, 'R': 1.0, 'max_position_size': 0.25}
    ),
    'Kalman Long-Short': (
        kalman_ma_crossover_long_short,
        {'Q_fast': 0.01, 'Q_slow': 0.001, 'R': 1.0, 'max_position_size': 0.25}
    )
}

# Run all strategies
results = {}
backtester = Backtester(config)

for name, (strategy_func, params) in strategies.items():
    print(f"\nRunning: {name}...")
    result = backtester.run(
        strategy=strategy_func,
        universe=universe,
        strategy_params=params
    )
    results[name] = result
    
print("\n✅ All strategies completed!")

In [ ]:
# Compare performance
comparison = pd.DataFrame({
    name: {
        'Final Value': result.equity_curve['total_value'].iloc[-1],
        'Total Return': (result.equity_curve['total_value'].iloc[-1] / 100000 - 1) * 100,
        'Num Trades': len(result.trades),
        'Avg Trade': result.equity_curve['total_value'].iloc[-1] / len(result.trades) if len(result.trades) > 0 else 0
    }
    for name, result in results.items()
}).T

comparison = comparison.round(2)
comparison['Final Value'] = comparison['Final Value'].apply(lambda x: f"${x:,.0f}")
comparison['Total Return'] = comparison['Total Return'].apply(lambda x: f"{x:.2f}%")
comparison['Num Trades'] = comparison['Num Trades'].astype(int)

print("\n📊 Strategy Comparison:")
print("=" * 70)
print(comparison)

## Example 3: Parameter Sweep

In [ ]:
# Test different MA periods
ma_combinations = [
    (10, 30),
    (20, 50),
    (50, 200)
]

sweep_results = {}
backtester = Backtester(config)

for fast, slow in ma_combinations:
    params = {
        'fast_ma': fast,
        'slow_ma': slow,
        'max_position_size': 0.25
    }
    
    result = backtester.run(
        strategy=ma_crossover_long_only,
        universe=universe,
        strategy_params=params
    )
    
    sweep_results[f"MA({fast},{slow})"] = result

# Display results
sweep_comparison = pd.DataFrame({
    name: {
        'Return %': (result.equity_curve['total_value'].iloc[-1] / 100000 - 1) * 100,
        'Num Trades': len(result.trades)
    }
    for name, result in sweep_results.items()
}).T

print("\n📊 MA Parameter Sweep:")
print("=" * 40)
print(sweep_comparison.round(2))

## Summary

**Benefits of Strategy Library:**
- ✅ Clean, reusable code
- ✅ Easy to compare strategies
- ✅ Simple parameter tuning
- ✅ Consistent implementation
- ✅ Quick notebook prototyping

**Next Steps:**
- Try different parameter combinations
- Add your own strategies to the library
- Compare against benchmarks
- Optimize with walk-forward analysis